## Setup

In [1]:
# import modules and set environment settings
import arcpy, os
arcpy.overwriteOutput = True

# set local variables
workspace =  "C:\\GIS\PROJECT\\ThresholdEvaluation\\ThresholdEvaluation.gdb"
sdeBase   = "C:\\GIS\\DB_CONNECT\\Vector.sde"

# sde datasets
parcels    = os.path.join(sdeBase,"sde.SDE.Parcels\sde.SDE.Parcel_Master")
soils      = os.path.join(sdeBase, "sde.SDE.Soils\sde.SDE.land_capability_NRCS2007") 
localplan  = os.path.join(sdeBase, "sde.SDE.Planning\sde.SDE.LocalPlan")
towncenter = os.path.join(sdeBase, "sde.SDE.Planning\sde.SDE.TownCenter")

# local impervious datasets
newImp = os.path.join(workspace, "Impervious_2018")
oldImp = os.path.join(workspace, "Impervious_2010")
                          
# sde impervious
impervious2010 = os.path.join(sdeBase, "sde.SDE.Tahoe_Impervious_2010//sde.SDE.Impervious_2010")                                                      
impervious2018 = os.path.join(sdeBase, "sde.SDE.Tahoe_Impervious_2010//sde.SDE.Impervious_2018")

# output tables
outTable2010 = "ImperviousOverlayAnalysis_2010.csv"
outTable2018 = "ImperviousOverlayAnalysis_2018.csv"
outLocation  = "C:\\GIS\PROJECT\\ThresholdEvaluation\\"

### 2010 Impervious Analysis

In [4]:
# output feature classes
id_NRCS_Impervious = "memory\id_NRCS_Impervious"

id_NRCS_Impervious_LocalPlan = "memory\id_NRCS_Impervious_LocalPlan"

id_NRCS_Impervious_LocalPlan_TownCenter = "memory\id_NRCS_Impervious_LocalPlan_TownCenter"

id_NRCS_Impervious_LocalPlan_TownCenter_Parcel = os.path.join(workspace,"id_NRCS_Impervious_LocalPlan_TownCenter_Parcel_2010")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious"):
    print("Deleting: memory\id_NRCS_Impervious")
    arcpy.Delete_management("memory\id_NRCS_Impervious")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan")
    
# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan_TownCenter"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan_TownCenter")

# delete staging feature class
if arcpy.Exists(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel):
    print("Deleting: id_NRCS_Impervious_LocalPlan_TownCenter_Parcel")
    arcpy.Delete_management(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)
    
# identify land capability/impervious
print("Identifying Soils with Impervious Surfaces...")
arcpy.Identity_analysis(soils, oldImp, id_NRCS_Impervious)

# identify land capability/impervious/local plan
print("Identifying Soils with Impervious Surfaces & Local Plan layers...")
arcpy.Identity_analysis(id_NRCS_Impervious, localplan, id_NRCS_Impervious_LocalPlan)

# identify land capability/impervious/local plan/town center
print("Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan, towncenter, id_NRCS_Impervious_LocalPlan_TownCenter)

# identify land capability/impervious/local plan/town center/parcels
print("Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan_TownCenter, parcels, id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)

# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it already exists
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTable2010)):
    arcpy.Delete_management(os.path.join(outLocation,outTable2010))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTable2010)

# success?
print("Table {0} created".format(outTable2010))

Deleting: memory\id_NRCS_Impervious
Deleting: memory\id_NRCS_Impervious_LocalPlan
Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter
Deleting: id_NRCS_Impervious_LocalPlan_TownCenter_Parcel
Identifying Soils with Impervious Surfaces...
Identifying Soils with Impervious Surfaces & Local Plan layers...
Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...
Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...
Calculating Acres...
Deleting Old Table...
Exporting New Table...
Table ImperviousOverlayAnalysis_2010.csv created


### 2018 Impervious Analysis

In [5]:
# output feature classes
id_NRCS_Impervious = "memory\id_NRCS_Impervious"

id_NRCS_Impervious_LocalPlan = "memory\id_NRCS_Impervious_LocalPlan"

id_NRCS_Impervious_LocalPlan_TownCenter = "memory\id_NRCS_Impervious_LocalPlan_TownCenter"

id_NRCS_Impervious_LocalPlan_TownCenter_Parcel = os.path.join(workspace,"id_NRCS_Impervious_LocalPlan_TownCenter_Parcel_2018")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious"):
    print("Deleting: memory\id_NRCS_Impervious")
    arcpy.Delete_management("memory\id_NRCS_Impervious")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan")
    
# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan_TownCenter"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan_TownCenter")

# delete staging feature class
if arcpy.Exists(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_Parcel")
    arcpy.Delete_management(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)
    
# identify land capability/impervious
print("Identifying Soils with Impervious Surfaces...")
arcpy.Identity_analysis(soils, newImp, id_NRCS_Impervious)

# identify land capability/impervious/local plan
print("Identifying Soils with Impervious Surfaces & Local Plan layers...")
arcpy.Identity_analysis(id_NRCS_Impervious, localplan, id_NRCS_Impervious_LocalPlan)

# identify land capability/impervious/local plan/town center
print("Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan, towncenter, id_NRCS_Impervious_LocalPlan_TownCenter)

# identify land capability/impervious/local plan/town center/parcels
print("Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan_TownCenter, parcels, id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)

# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it already exists
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTable2018)):
    arcpy.Delete_management(os.path.join(outLocation,outTable2018))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTable2018)

# success?
print("Table {0} created".format(outTable2018))

Deleting: memory\id_NRCS_Impervious
Deleting: memory\id_NRCS_Impervious_LocalPlan
Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter
Identifying Soils with Impervious Surfaces...
Identifying Soils with Impervious Surfaces & Local Plan layers...
Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...
Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...
Calculating Acres...
Deleting Old Table...
Exporting New Table...
Table ImperviousOverlayAnalysis_2018.csv created


## Coverage Analysis
### Recreate these tables: 
#### F:\Research and Analysis\Reporting\Annual Reports\2016 Annual Report\Data and Analysis\Coverage\RawData\AnalysisProduct

#### F:\Research and Analysis\Reporting\Annual Reports\2016 Annual Report\Data and Analysis\Coverage\RawData


In [3]:
# import modules and set environment settings
import arcpy, os
arcpy.overwriteOutput = True

# set local variables
workspace = 
sdeBase = "F:/GIS/GIS_DATA/Vector.sde"

soils = sdeBase + "//sde.SDE.Soils//sde.SDE.land_capability_NRCS2007" 
localplan = sdeBase + "//sde.SDE.Planning//sde.SDE.LocalPlan"
# towncenter = sdeBase + "//sde.SDE.Planning//sde.SDE.TownCenter"

# impervious datasets
newImp = os.path.join(workspace, "Impervious_2018_2019")
oldImp = os.path.join(workspace, "Impervious_2010_Cleaned")

# impervious = sdeBase + "//sde.SDE.Tahoe_Impervious_2010//sde.SDE.Impervious_2010"

# current parcels
parcels = sdeBase + "//sde.SDE.Parcels//sde.SDE.Parcel_Master"

# tables
# outTableImpervious = "id_NRCS_LocalPlan_TownCenter_Impervious.csv"
outTableParcel = "id_NRCS_LocalPlan_Impervious_Parcel.csv"
outLocation = r"F:/Research and Analysis/Reporting/Annual Reports/2017 Annual Report/2017-Annual-Report_Data/Coverage/rawdata"
fgdb = "F:\\Research and Analysis\\Reporting\\Annual Reports\\2017 Annual Report\\2017-Annual-Report_Data\\Coverage\\rawdata\\Coverage.gdb"

# output feature classes
id_NRCS_LocalPlan = "in_memory/id_NRCS_LocalPlan"
# id_NRCS_LocalPlan_TownCenter = "in_memory/id_NRCS_LocalPlan_TownCenter"
id_NRCS_LocalPlan_Impervious = "in_memory/id_NRCS_LocalPlan_Impervious"
id_NRCS_LocalPlan_Impervious_Parcel = fgdb + "/id_NRCS_LocalPlan_Impervious_Parcel"

# delete in memory feature classes
if arcpy.Exists("in_memory/id_NRCS_LocalPlan"):
    print("Deleting: in_memory/id_NRCS_LocalPlan")
    arcpy.Delete_management("in_memory/id_NRCS_LocalPlan")

# if arcpy.Exists("in_memory/id_NRCS_LocalPlan_TownCenter"):
#     print("Deleting: in_memory/id_NRCS_LocalPlan_TownCenter")
#     arcpy.Delete_management("in_memory/id_NRCS_LocalPlan_TownCenter")
    
if arcpy.Exists("in_memory/id_NRCS_LocalPlan_Impervious"):
    print("Deleting: in_memory/id_NRCS_LocalPlan_Impervious")
    arcpy.Delete_management("in_memory/id_NRCS_LocalPlan_Impervious")
    
if arcpy.Exists(fgdb + "//" + "id_NRCS_LocalPlan_Impervious_Parcel"):
    print("Deleting: id_NRCS_LocalPlan_Impervious_Parcel")
    arcpy.Delete_management(fgdb + "//" + "id_NRCS_LocalPlan_Impervious_Parcel")

# identify land capability/local plan 
print("Identifying Soils with Local Plan...")
arcpy.Identity_analysis(soils, localplan, id_NRCS_LocalPlan)

# # identify land capability/local plan/town center
# print("Identifying Soils and Local Plan with Town Center...")
# arcpy.Identity_analysis(id_NRCS_LocalPlan, towncenter, id_NRCS_LocalPlan_TownCenter)

# identify land capability/local plan/impervious
print("Identifying Soils,and Local Plan with Impervious Surfaces...")
arcpy.Identity_analysis(id_NRCS_LocalPlan, impervious, id_NRCS_LocalPlan_Impervious)

## Export Table without Parcels baked in:

# # calculate acres
# with arcpy.da.UpdateCursor(id_NRCS_LocalPlan_TownCenter_Impervious, ['GISAcre', 'SHAPE@']) as cursor:
#     for row in cursor:
#         row[0] = row[1].getArea('GEODESIC', 'ACRES')
#         cursor.updateRow(row)

# # delete table if it exists
# if arcpy.Exists(os.path.join(outLocation,outTableTownCenterImpervious)):
#     arcpy.Delete_management(os.path.join(outLocation,outTableTownCenterImpervious))
    
# # Execute Table To Table
# arcpy.TableToTable_conversion(id_NRCS_LocalPlan_Impervious, outLocation, outTableImpervious)

# # success?
# print("Table {0} created".format(outTableImpervious))

# identify land capability/local plan/impervious/parcel master
print("Identifying Soils,Local Plan, and Impervious Surfaces with Parcels...")
arcpy.Identity_analysis(id_NRCS_LocalPlan_Impervious, parcels, id_NRCS_LocalPlan_Impervious_Parcel)

# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(id_NRCS_LocalPlan_Impervious_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it already exists
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTableParcel)):
    arcpy.Delete_management(os.path.join(outLocation,outTableParcel))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_LocalPlan_Impervious_Parcel, outLocation, outTableParcel)

# success?
print("Table {0} created".format(outTableParcel))

Identifying Soils with Local Plan...
Identifying Soils,and Local Plan with Impervious Surfaces...
Identifying Soils,Local Plan, Town Center, and Impervious Surfaces with Parcels...
Calculating Acres...
Deleting Old Table...
Exporting New Table...
Table id_NRCS_LocalPlan_Impervious_Parcel.csv created


### References
#### https://pro.arcgis.com/en/pro-app/tool-reference/analysis/identity.htm
#### https://pro.arcgis.com/en/pro-app/arcpy/functions/exists.htm
#### https://pro.arcgis.com/en/pro-app/arcpy/data-access/updatecursor-class.htm
#### https://pro.arcgis.com/en/pro-app/tool-reference/conversion/table-to-table.htm